In [ ]:
import os

from langchain_community.graphs import Neo4jGraph


os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "tmu-2024"

graph = Neo4jGraph()

In [ ]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
import langchain 
langchain.debug = True

# llm = ChatOllama(model="llama3:8b-instruct-q8_0", temperature=0)
llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

llm_transformer = LLMGraphTransformer(llm=llm)

In [ ]:
# get review text from duckdb
import duckdb
con = duckdb.connect('/home/kchauhan/repos/mds-tmu-mrp/db/duckdb/amazon_reviews.duckdb')

result= con.execute(
    """SELECT  b.title, b.parent_asin, a.title as review_title, a.text as review_text, a.helpful_vote
    FROM raw_review_Books  a
                    INNER JOIN raw_meta_Books b
                    ON a.parent_asin = b.parent_asin
                    WHERE CAST(a.helpful_vote AS INT) > 0
                    LIMIT 100
                    """).fetchall()

for row in result:
    print(row)
    print("")


In [ ]:
from langchain_core.documents import Document

text = """
The Silent Hill series of video games is one of the best of its genre. With the exception of SH 3, which has more of a blockbustery feel to it, the original game and its sequel, Silent Hill 2, have done more to further the genre of Survival Horror than any other series of games, making Capcom's Resident Evil/Biohazard series look slow and tired.<br />We start off with James Sunderland, a shell of a man since his wife Mary's death, three years before, lost in thought in a bathroom on the outskirts of town. He's come back to SIlent Hill, a resort town full of happy memories for himself and his wife, because he has received a letter from her, saying she will meet him in their 'special place'. And from there, you're pretty much left to piece the story together yourself, thanks to ingenious plot twists and the best characterisation since Final Fantasy 7.<br />The graphics are wonderful. Fuzzy, murky streets and rooms harbour beautifully-rendered, totally ambiguous creatures that scare us becuase of what we cannot see. The superhuman, exaggerated animation of the mannequins and nurses add to the unnatural atmosphere pervading the game. The cut-scene renders are actually beautiful, with Maria and James' exchange in the underground Toluca Prison levels being one of the most riveting pieces of FMV ever. The sound, too, adds greatly to the experience, with grunts, bangs and screams punctuating the sometimes-deafening silence. Because of the game mechanics, what you hear is oftentimes more important than what you see (or believe you see) and it's this, coupled with the storyline and characters, that characterises the unique psychological tension found in this game.<br />The game mechanics are simple to come to terms with and prove themselves after fifteen minutes or so of play time: you don't notice them anymore.<br />A word on what makes SH 2 such a wonderful experience - Characters like Laura, an enigmatic child who claims to know Mary, Eddie Dombrowski, an unhinged giant of a man whose attitude towards cold-blooded murder gives a realistic slant to the psychological horror, and Maria, the sensual and schizophrenic doppelganger of James' dead wife, are so well-realised that we see new characteristics in them each time the game is played. The character of Angela is a particular mystery, and a testament to the strength of emotion that a videogame can create. On paper, the main villian, Pyramid Head, is nothing special (a giant man with a steel pyramid for a head... literally!) but it's the way he moves and the noises he makes that propel him into the history books as one of the scariest villians anywhere, alongside Biohazard's Hunters and The Exorcist's Pazuzu, in terms of the amount of fear he generates. This is the genius of the game, Konami's developers taking painstaking time and effort with the people that populate the terrifying world of Silent Hill, thereby giving us an unforgettable and compelling cast of characters.<br />The replay value is very high, with five different endings to discover, and new things to see and hear each time, every trip you make to Silent Hill 2 leaves you with something new from this bizarre and macabre universe.<br />A word for fans of action-based violence: you'll be disappointed. SH 2 is a story-based, character-driven game, and if it's violence and gore you need, then this won't do. It's a different kind of scary than Resident Evil, Clock Tower etc. For anyone interested in seeing just how far a game can push the concept of psychological horror, or for anyone who loves to get lost in a storyline, this is one of the best examples you can own.<br />Put simply, Silent Hill 2 is an Opus, a magnificent monster of a game that should held up as a standard-bearer by more than just the videogames industry. It's deep, it's complex, it's engaging, and above all, it's excellent. Thoroughly and unequivocally recommended."""
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)

print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

In [ ]:
from langchain_core.documents import Document

for row in result:
    text = row[3]
    print (text)
    documents = [Document(page_content=text)]
    graph_documents = llm_transformer.convert_to_graph_documents(documents)

    # print(f"Nodes:{graph_documents[0].nodes}")
    # print(f"Relationships:{graph_documents[0].relationships}")
    
    # Convert nodes and relationships to json
    nodes = []
    for node in graph_documents[0].nodes:
        nodes.append(str(node))
    relationships = []
    for relationship in graph_documents[0].relationships:
        relationships.append(str(relationship))
    print(f"Nodes:{nodes}")
    print(f"Relationships:{relationships}")
    
    # write final json with nodes and relationships to file 
    import json
    with open(f"output/gpt-4o-mini/{row[1]}.json", "w") as f:
        json.dump( {"title": row[0], "parent_asin": row[1], "review_title": row[2], "review_text": row[3], "helpful_vote": row[4], "nodes": nodes, "relationships": relationships}, f)
    

In [ ]:
graph.add_graph_documents(graph_documents)